# Episodio 12: definizione e istanziamento di typeclass, parte 1
Gli esempi qui sotto possono essere eseguiti copiandoli nell'interprete di Haskell, il `ghci`. Se vuoi eseguire direttamente questo notebook, le istruzioni su come farlo sono disponibili a [questo](https://github.com/gibiansky/IHaskell) link.

Nella scorsa lezione, abbiamo istanziato una typeclass preesistente, `Show`. Abbiamo anche visto la sintassi per definire nuove typeclass. In questa lezione vedremo un esempio pratico di definizione ed istanziamento di una typeclass non standard, ovvero di una typeclass che non è definita sotto altro nome in nessuna delle librerie di Haskell di uso comune.
L'esempio è fortemente ispirato alla [typeclass `YesNo` di _Learn You A Haskell, For Great Good!_](http://learnyouahaskell.com/making-our-own-types-and-typeclasses#a-yes-no-typeclass), che prende spunto a sua volta da alcune cose che si possono fare in linguaggi debolmente tipizzati come il JavaScript e il Python, ma che il fortissimo type system di Haskell non permette.

## La caratteristica del Python che cercheremo di imitare
In Python, è possibile scrivere condizionali come il seguente:

```python
if stringa:
    print(stringa)
else:
    print("mi dispiace, la stringa è vuota!")
```

Il comportamento di questo codice varia a seconda che la stringa sia vuota (`""`) o "piena" (per es. `"ciao"`). Nel primo il risultato sarà un bel `mi dispiace, la stringa è vuota!`, mentre se la stringa non è vuota ne verrà visualizzato il contenuto.
Questo significa che, in un `if`, le stringhe vengono trattate un po' come booleani: se vuote, vengono considerate "false", altrimenti "vere".

Questo ragionamento si applica non solo alle stringhe, ma a quasi tutti i tipi: numeri, liste, dizionari...

## Una typeclass "booleaneggiante" in Haskell
Con la premessa che il rendere confrontabili booleani con tipi arbitrari é a mio avviso un'idea __nefasta__, cercheremo di replicare questa feature in Haskell tramite una typeclass, che chiameremo `Booleaneggiante`. Questa typeclass avrà due funzioni, `veruccio` e `falsuccio`, definite l'una in termini dell'altra in modo che, all'istanziamento, sia sufficiente specificare il comportamento di solo una delle due per poter ottenere automaticamente anche l'altra:

In [23]:
class Booleaneggiante a where
    veruccio :: a -> Bool
    veruccio a = not (falsuccio a)

    falsuccio :: a -> Bool
    falsuccio a = not (veruccio a) 

La variabile di tipo `a`, senza alcun constraint, significa che qualunque tipo può essere istanza di `Booleaneggiante`.
Il "default" di `veruccio` è definito come l'opposto di `falsuccio` e viceversa.

## Istanziamento di `Booleaneggiante`

### Bool
Per prima cosa, ovviamente, rendiamo `Bool` istanza di `Booleaneggiante`:

In [24]:
instance Booleaneggiante Bool where
    veruccio True = True
    veruccio False = False

Facile, no? Come anticipato, non é necessario definire anche `falsuccio` per via della definizione che abbiamo scritto all'interno della typeclass stessa. Difatti possiamo scrivere:

In [25]:
falsuccio True

False

In [26]:
falsuccio False

True

Passiamo ora ad istanziare la nostra nuova typeclass per altri tipi di dato.

### Int

In [27]:
instance Booleaneggiante Int where
    veruccio 0 = False
    veruccio _ = True

In [28]:
n = 0 :: Int
n' = 15 :: Int

In [29]:
veruccio n

False

In [30]:
veruccio n'

True

### Liste

In [31]:
instance Booleaneggiante [a] where
    veruccio [] = False
    veruccio _ = True

In [32]:
veruccio []

False

In [33]:
veruccio [1,2,3]

True

Ovviamente, questo fa sì che `veruccio` e `falsuccio` funzionino anche per le stringhe, che sono liste di caratteri. Di conseguenza, possiamo replicare l'esempio che abbiamo visto all'inizio in Python:

In [34]:
esempioPython stringa = 
    if veruccio stringa -- particamente basta aggiungere un "veruccio" qui
        then putStrLn stringa
        else putStrLn "mi dispiace, la stringa è vuota!"

In [35]:
esempioPython "ciao"

ciao

In [36]:
esempioPython ""

mi dispiace, la stringa è vuota!

### Maybe

In [37]:
instance Booleaneggiante (Maybe a) where
    veruccio Nothing = False
    veruccio _ = True

In [38]:
veruccio Nothing

False

In [39]:
veruccio (Just "ciao")

True

In [40]:
falsuccio Nothing

True

In [41]:
falsuccio (Just "ciao")

False